# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = list(get_files("data/song_data"))
print(song_files)

['/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json', '/home/workspace/data/song_data/A/B/B/TRABBOR128F4286200.json', '/home/workspace/data/song_data/A/B/B/TRABBOP128F931B50D.json', '/home/workspace/data/song_data/A/B/B/TRABBNP128F932546F.json', '/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json', '/home/workspace/data/song_data/A/B/B/TRABBTA128F933D304.json', '/home/workspace/data/song_data/A/B/B/TRABBKX128F4285205.json', '/home/workspace/data/song_data/A/B/B/TRABBAM128F429D223.json', '/home/workspace/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json', '/home/workspace/data/song_data/A/B/B/TRABBJE12903CDB442.json', '/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json', '/home/workspace/data/song_data/A/B/B/TRABBZN12903CD9297.json', '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json', '/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json', '/home/workspace/data/song_data/A/B/C/T

In [5]:
filepath = song_files[0] 
print(filepath)

/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json


In [6]:

df = pd.read_json(filepath, lines=True)
df.head()
#df.dtypes

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
df_selected = df[['song_id', 'title', 'artist_id', 'year', 'duration']]

song_data_array = df_selected.values[0]

song_data = song_data_array.tolist()

song_data


['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

In [13]:
print(song_data)

['SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
df_artist = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']] 

#artist_data_array = df_artist.values[0]
artist_data_array = df_artist.values

artist_data = artist_data_array.tolist()
#artist_data

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = list(get_files("data/log_data"))

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df_filter = df[(df.page == "NextSong")]
df_filter.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [15]:
#df['t'] = pd.to_datetime(df['ts'])
t = pd.to_datetime(df_filter['ts'])
t.head()
#print(t[0])

0   1970-01-01 00:25:43.449657796
1   1970-01-01 00:25:43.449690796
2   1970-01-01 00:25:43.449841796
3   1970-01-01 00:25:43.449895796
4   1970-01-01 00:25:43.450033796
Name: ts, dtype: datetime64[ns]

In [16]:
time_extract = [t, t.dt.time, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]

#time_extract = [t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]
 
column_labels = ['timestamp','time','hour', 'day', 'week of year', 'month', 'year', 'weekday']


In [17]:
time_dict = dict(zip(column_labels, time_extract))

In [18]:
#time_data = (t.dt.year)
#column_labels = ()

In [19]:
time_df = pd.DataFrame(time_dict)
time_df.head()


,timestamp,time,hour,day,week of year,month,year,weekday
0,1970-01-01 00:25:43.449657796,00:25:43.449657,0,1,1,1,1970,3
1,1970-01-01 00:25:43.449690796,00:25:43.449690,0,1,1,1,1970,3
2,1970-01-01 00:25:43.449841796,00:25:43.449841,0,1,1,1,1970,3
3,1970-01-01 00:25:43.449895796,00:25:43.449895,0,1,1,1,1970,3
4,1970-01-01 00:25:43.450033796,00:25:43.450033,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
# user_id, first_name, last_name, gender, level
user_df = df[['userId', 'firstName', 'lastName' , 'gender', 'level']].drop_duplicates()
#user_df.drop_duplicates()
user_df.head()


,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
21,50,Ava,Robinson,F,free
35,54,Kaleb,Cook,M,free
40,32,Lily,Burns,F,free


In [22]:
#user_df[(user_df.userid == 24)]
#user_df


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [34]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None     
       
    key = (str(row.itemInSession) + str(row.sessionId))
   
    # insert songplay record artist auth firstName gender itemInSession lastName length level location method page registration sessionId song status ts userAgent userId
    songplay_data = (key, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
   
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

53 954
53954
88 984
88984
89 984
89984
54 954
54954
90 984
90984
91 984
91984
55 954
55954
56 954
56954
92 984
92984
93 984
93984
94 984
94984
95 984
95984
96 984
96984
97 984
97984
98 984
98984
99 984
99984
100 984
100984
101 984
101984
102 984
102984
103 984
103984
104 984
104984
0 1011
01011
105 984
105984
106 984
106984
107 984
107984
108 984
108984
109 984
109984
110 984
110984
111 984
111984
112 984
112984
113 984
113984
114 984
114984
115 984
115984
116 984
116984
117 984
117984
0 951
0951
1 951
1951
2 951
2951
3 951
3951
4 951
4951
0 1045
01045
1 1045
11045
0 587
0587
0 990
0990
1 990
1990
2 990
2990
0 1038
01038
1 1038
11038
0 865
0865
1 865
1865
0 1053
01053
0 1016
01016
1 1016
11016
0 931
0931
2 1016
21016
3 1016
31016
0 1005
01005
0 1055
01055
0 947
0947
0 1001
01001
0 1041
01041
0 1040
01040
1 1040
11040
1 1041
11041
2 1041
21041
3 1041
31041
4 1041
41041
5 1041
51041
6 1041
61041
7 1041
71041
8 1041
81041
9 1041
91041
10 1041
101041
11 1041
111041
12 1041
121041
13 1041
1

In [33]:
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.